In [2]:
from collections import Counter
import glob
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import math
from scipy import optimize
#from ipywidgets import interact
#import ipywidgets as wg
import statistics as st

# Funções para carregar vetores com hits e PMTs

In [3]:
#Função para carregar um vetor de hits / PMT 
def fnt_array_hitspmt(list_sum):
    xf = []
    for filetr in list_sum:

         with open(filetr) as fo:
            s_energy = fo.readline().replace(",",".")
            i_Event = int(s_energy[0:7])
            i_PMT = int(s_energy[15:21])
            i_Hits = int(s_energy[8:14])
            if (i_PMT != 0):
                xf.append(float(i_Hits / i_PMT))
        
            while (s_energy != ""):
                s_energy = fo.readline().replace(",",".")
                if (s_energy != ""):
                    i_Event = int(s_energy[0:7])
                    i_PMT = int(s_energy[15:21])
                    i_Hits = int(s_energy[8:14])
                    if (i_PMT != 0):
                        xf.append(float(i_Hits / i_PMT))
    fo.close()
    return(xf)

#Função para carregar um vetor de dois vetores: 1) hits; 2) PMT; 3) Energia
#novo parametro trigger_pmt para somente considerar eventos com multiplicidade >=trigger_pmt
#a trigger no detector somente ativa as NDAQs se mais que 5 PMTs fizeram a detecção - 31/05/2020
#valor padrão trigger_pmt=0 para considerar tudo
def fnt_array_vet_hits_pmt(list_sum,trigger_pmt=0):
    xf = []
    yf = []
    zf = []
    for filetr in list_sum:

        with open(filetr) as fo:
            s_energy = fo.readline().replace(",",".")
            i_PMT = int(s_energy[15:21])
            i_Hits = int(s_energy[8:14])
            f_Energy = ( i_Hits * 77.96 ) * 4.9E-4 + 0.542 #float(s_energy[43:63])
            #DUQ/hits = 77.96 e Energia(MeV) = Hits * (DUQ/hits) * 4.9E-4 + 0.542 
            #i_HitsPMT = float(i_Hits / i_HitsPMT
            if (i_Hits != 0)and(i_PMT>=trigger_pmt)and(i_Hits<=590):
                yf.append(i_PMT)
                xf.append(i_Hits)
                zf.append(f_Energy)
        
            while (s_energy != ""):
                s_energy = fo.readline().replace(",",".")
                if (s_energy != ""):
                    i_PMT = int(s_energy[15:21])
                    #print(energy_histo)
                    i_Hits = int(s_energy[8:14])
                    f_Energy = ( i_Hits * 77.96 ) * 4.9E-4 + 0.542 
                    #print(i_Hits,fEner)
                    #DUQ/hits = 77.96 e Energia(MeV) = Hits * (DUQ/hits) * 4.9E-4 + 0.542 
                    #i_HitsPMT = int(s_energy[8:14])
                    if (i_Hits != 0)and(i_PMT>=trigger_pmt)and(i_Hits<=590):
                        yf.append(i_PMT)
                        xf.append(i_Hits)
                        zf.append(f_Energy)
    fo.close()
    return( [xf, yf, zf] )

#Função para carregar um vetor de dois vetores: 1) hits; 2) PMT 
def fnt_array_vet_hits_pmt_regime(list_sum, regime1, regime2):
    xf = []
    yf = []
    for filetr in list_sum:

        with open(filetr) as fo:
            s_energy = fo.readline().replace(",",".")
            i_PMT = int(s_energy[15:21])
            i_Hits = int(s_energy[8:14])
            #i_HitsPMT = float(i_Hits / i_HitsPMT
            if (i_Hits != 0)and(i_PMT>regime1)and(i_PMT<=regime2):
                yf.append(i_PMT)
                xf.append(i_Hits)
        
            while (s_energy != ""):
                s_energy = fo.readline().replace(",",".")
                if (s_energy != "")and(i_PMT>regime1)and(i_PMT<=regime2):
                    i_PMT = int(s_energy[15:21])
                    #print(energy_histo)
                    i_Hits = int(s_energy[8:14])
                    #i_HitsPMT = int(s_energy[8:14])
                    if (i_Hits != 0):
                        yf.append(i_PMT)
                        xf.append(i_Hits)
    fo.close()
    return( [xf, yf] )

#Função para carregar um vetor de hits / PMT  com limite maior e menor de uma determinada multiplicidade de PMT
def fnt_array_hitspmt_limits(list_sum, check_limit_PMT):
    xf = [] #menor multiplicidade
    yf = [] #maior multiplicidade
    for filetr in list_sum:
         with open(filetr) as fo:
            s_energy = fo.readline().replace(",",".")
            i_Event = int(s_energy[0:7])
            i_PMT = int(s_energy[15:21])
            i_Hits = int(s_energy[8:14])
#            fg = open(list_gama[0],'r')
#            s_gama = fg.readline().replace(",",".")
            if (i_PMT != 0):
                if (i_PMT > check_limit_PMT):
                    yf.append(float(i_Hits / i_PMT))
#                    print('Multiplicidade > '+str(check_limit_PMT)+' = '+s_gama)
                else:
                    xf.append(float(i_Hits / i_PMT))
#                    print('Multiplicidade < '+str(check_limit_PMT)+' = '+s_gama)
        
            while (s_energy != ""):
                s_energy = fo.readline().replace(",",".")
#                s_gama = fg.readline().replace(",",".")
                if (s_energy != ""):
                    i_Event = int(s_energy[0:7])
                    i_PMT = int(s_energy[15:21])
                    i_Hits = int(s_energy[8:14])
                    if (i_PMT != 0):
                        if (i_PMT > check_limit_PMT):
                            yf.append(float(i_Hits / i_PMT))
#                            print('Multiplicidade > '+str(check_limit_PMT)+' = '+s_gama)
                        else:
                            xf.append(float(i_Hits / i_PMT))
#                            print('Multiplicidade < '+str(check_limit_PMT)+' = '+s_gama)
    fo.close()
    return([xf,yf])

#Função para carregar um vetor de hits / PMT  com limite maior e menor de uma determinada multiplicidade de PMT
#O diferencial desta função é que aqui estou comparando energia por multiplicidade
def fnt_array_hitspmt_limits_2(list_sum, check_limit_PMT, list_gama):
    xf = [] #menor multiplicidade
    yf = [] #maior multiplicidade
    xg = []
    yg = []
    af = []
    bf = []
    for filetr in list_sum:
         with open(filetr) as fo:
            s_energy = fo.readline().replace(",",".")
            i_Event = int(s_energy[0:7])
            i_PMT = int(s_energy[15:21])
            i_Hits = int(s_energy[8:14])
            fg = open(list_gama[0],'r')
            s_gama = fg.readline().replace(",",".")
            f_gama = float(s_energy[22:36])  #para converter em MeV
            if (i_PMT != 0):
                if (i_PMT > check_limit_PMT):
                    yf.append(float(i_Hits / i_PMT))
                    yg.append(float(s_gama[37:]))
                    bf.append(f_gama)
                    #print('Multiplicidade > '+str(check_limit_PMT)+' = '+s_gama)
                else:
                    xf.append(float(i_Hits / i_PMT))
                    xg.append(float(s_gama[37:]))
                    af.append(f_gama)
                    #print('Multiplicidade < '+str(check_limit_PMT)+' = '+s_gama)
        
            while (s_energy != ""):
                s_energy = fo.readline().replace(",",".")
                s_gama = fg.readline().replace(",",".")
                if (s_energy != ""):
                    i_Event = int(s_energy[0:7])
                    i_PMT = int(s_energy[15:21])
                    i_Hits = int(s_energy[8:14])
                    f_gama = float(s_energy[22:36])  #para converter em MeV
                    if (i_PMT != 0):
                        if (i_PMT > check_limit_PMT):
                            yf.append(float(i_Hits / i_PMT))
                            yg.append(float(s_gama[37:]))
                            bf.append(f_gama)
                            #print('Multiplicidade > '+str(check_limit_PMT)+' = '+s_gama)
                        else:
                            xf.append(float(i_Hits / i_PMT))
                            xg.append(float(s_gama[37:]))
                            af.append(f_gama)
                            #print('Multiplicidade < '+str(check_limit_PMT)+' = '+s_gama)
    fo.close()
    return([xf,yf,xg,yg,af,bf])

#Função para carregar um vetor de hits / PMT  com limite maior e menor de uma determinada multiplicidade de PMT
#O diferencial desta função é que aqui estou comparando energia por multiplicidade
def fnt_array_hitspmt_limits_3(list_sum, check_limit_PMT):
    xf = [] #menor multiplicidade
    yf = [] #maior multiplicidade
    xg = []
    yg = []
    for filetr in list_sum:
         with open(filetr) as fo:
            s_energy = fo.readline().replace(",",".")
            i_Event = int(s_energy[0:7])
            i_PMT = int(s_energy[15:21])
            i_Hits = int(s_energy[8:14])
            #fg = open(list_gama[0],'r')
            #f_gama = float(s_energy[22:41])*1000  #para converter em MeV
            f_gama = float(s_energy[22:36])  #para converter em MeV
            if (i_PMT != 0):
                if (i_PMT > check_limit_PMT):
                    yf.append(float(i_Hits / i_PMT))
                    yg.append(f_gama)
                    #print('Multiplicidade > '+str(check_limit_PMT)+' = '+s_gama)
                else:
                    xf.append(float(i_Hits / i_PMT))
                    xg.append(f_gama)
                    #print('Multiplicidade < '+str(check_limit_PMT)+' = '+s_gama)
        
            while (s_energy != ""):
                s_energy = fo.readline().replace(",",".")
                #s_gama = fg.readline().replace(",",".")
                if (s_energy != ""):
                    i_Event = int(s_energy[0:7])
                    i_PMT = int(s_energy[15:21])
                    i_Hits = int(s_energy[8:14])
                    #f_gama = float(s_energy[22:41])*1000 #para converter em MeV
                    f_gama = float(s_energy[22:36]) #para converter em MeV
                    if (i_PMT != 0):
                        if (i_PMT > check_limit_PMT):
                            yf.append(float(i_Hits / i_PMT))
                            yg.append(f_gama)
                            #print('Multiplicidade > '+str(check_limit_PMT)+' = '+s_gama)
                        else:
                            xf.append(float(i_Hits / i_PMT))
                            xg.append(f_gama)
                            #print('Multiplicidade < '+str(check_limit_PMT)+' = '+s_gama)
    fo.close()
    return([xf,yf,xg,yg])

#Função para carregar um vetor de dois vetores: 1) hits; 2) PMT; 3) Energia Depositada; 4) Energia coletada
def fnt_array_vet_hits_energy(list_sum):
    xf = []
    yf = []
    chf = []
    ehf = []
    for filetr in list_sum:

        with open(filetr) as fo:
            s_energy = fo.readline().replace(",",".")
            i_PMT = int(s_energy[15:21])
            i_Hits = int(s_energy[8:14])
            f_Cherenkov = float(s_energy[22:42])*1000
            f_EnergyHit = float(s_energy[43:63])*1000
            if (i_Hits != 0):
                yf.append(i_PMT)
                xf.append(i_Hits)
                chf.append(f_Cherenkov)
                ehf.append(f_EnergyHit)
        
            while (s_energy != ""):
                s_energy = fo.readline().replace(",",".")
                if (s_energy != ""):
                    i_PMT = int(s_energy[15:21])
                    i_Hits = int(s_energy[8:14])
                    f_Cherenkov = float(s_energy[22:42])*1000
                    f_EnergyHit = float(s_energy[43:63])*1000
                    if (i_Hits != 0):
                        yf.append(i_PMT)
                        xf.append(i_Hits)
                        chf.append(f_Cherenkov)
                        ehf.append(f_EnergyHit)
    fo.close()
    return( [xf, yf, chf, ehf] )

#Função para carregar um vetor de hits / PMT 
def fnt_array_hitspmt_veto(list_sum):
    ev = []
    xf = []
    yf = []
    chf = []
    ehf = []
    vxf = []
    vyf = []
    for filetr in list_sum:

        with open(filetr) as fo:
            s_energy = fo.readline().replace(",",".")
            i_Event = int(s_energy[1:7])
            i_Hits = int(s_energy[8:14])
            i_PMT = int(s_energy[15:21])
            f_Cherenkov = float(s_energy[22:42])*1000
            f_EnergyHit = float(s_energy[43:63])*1000
            i_Hits_Veto = int(s_energy[64:70])
            i_PMT_Veto = int(s_energy[71:77])
            ev.append(i_Event)
            xf.append(i_Hits)
            yf.append(i_PMT)
            chf.append(f_Cherenkov)
            ehf.append(f_EnergyHit)
            vxf.append(i_Hits_Veto)
            vyf.append(i_PMT_Veto)
        
            while (s_energy != ""):
                s_energy = fo.readline().replace(",",".")
                if (s_energy != ""):
                    i_Event = int(s_energy[1:7])
                    i_PMT = int(s_energy[15:21])
                    i_Hits = int(s_energy[8:14])
                    f_Cherenkov = float(s_energy[22:42])*1000
                    f_EnergyHit = float(s_energy[43:63])*1000
                    i_Hits_Veto = int(s_energy[64:70])
                    i_PMT_Veto = int(s_energy[71:77])
                    ev.append(i_Event)
                    xf.append(i_Hits)
                    yf.append(i_PMT)
                    chf.append(f_Cherenkov)
                    ehf.append(f_EnergyHit)
                    vxf.append(i_Hits_Veto)
                    vyf.append(i_PMT_Veto)
    fo.close()
    return( [ev, xf, yf, chf, ehf, vxf, vyf] )



In [2]:
def fnt_array_scatter_hits(list_sum):
    xf = []
    yf = []
    for filetr in list_sum:
        with open(filetr) as fo:
            s_energy = fo.readline().replace(",",".")
            i_PMT = int(s_energy[15:21])
            i_Hits = int(s_energy[8:14])
            if (i_Hits != 0):
                xf.append(i_PMT)
                yf.append(i_Hits)
        
            while (s_energy != ""):
                s_energy = fo.readline().replace(",",".")
                if (s_energy != ""):
                    i_PMT = int(s_energy[15:21])
                    #print(energy_histo)
                    i_Hits = int(s_energy[8:14])
                    if (i_Hits != 0):
                        xf.append(i_PMT)
                        yf.append(i_Hits)
        fo.close()

    return([xf,yf])

In [6]:
%run def_functions_fits.ipynb

def fnt_array_IBD_Time(list_sum):
    xf = []
    pi = []
    di = []
    pf = []
    df = []
    mf = []
    mp = []
    md = []
    nf = []
    nr_eventos = 0
    for filetr in list_sum:
        with open(filetr) as fo:
            s_IBD = fo.readline().replace(",",".")
            #print(s_IBD[71:91])
            nr_eventos += 1
            f_Time_ns = float(s_IBD[71:91].strip())
            i_Prompt_pe = int(s_IBD[8:14].strip())
            i_Delay_pe = int(s_IBD[36:42].strip())
            f_Prompt_pe = fnt_pe2MeV(float(s_IBD[8:14].strip()))
            f_Delay_pe = fnt_pe2MeV(float(s_IBD[36:42].strip()))
            i_Multiplicidade = int(s_IBD[64:70].strip())
            i_Multip_prompt = int(s_IBD[92:98].strip())
            i_Multip_delay = int(s_IBD[99:105].strip())
            if ((f_Time_ns >= 0)and(i_Prompt_pe > 0)
               and(i_Delay_pe > 0)and(i_Multiplicidade > 0)
                and(f_Delay_pe < 20)):
                xf.append(f_Time_ns)
                pi.append(i_Prompt_pe)
                di.append(i_Delay_pe)
                pf.append(f_Prompt_pe)
                df.append(f_Delay_pe)
                mf.append(i_Multiplicidade)
                mp.append(i_Multip_prompt)
                md.append(i_Multip_delay)
            if ((f_Time_ns >= 8000)and(f_Time_ns <= 50000)
              and(f_Prompt_pe >= 3)and(f_Prompt_pe <= 10)
                and(f_Delay_pe >= 1.6)and(f_Delay_pe <= 7.6)
                and(i_Multip_delay >= 25)):
                nf.append(fnt_Positron2Neutrino(f_Prompt_pe))
        
            while (s_IBD.strip() != ""):
                s_IBD = fo.readline().replace(",",".")
                if (s_IBD.strip() == ""):
                    break
                nr_eventos += 1
                #print('IBD: '+s_IBD)
                #print('IBD p: '+s_IBD[8:14].strip())
                f_Time_ns = float(s_IBD[71:91].strip())
                i_Prompt_pe = int(s_IBD[8:14].strip())
                i_Delay_pe = int(s_IBD[36:42].strip())
                f_Prompt_pe = fnt_pe2MeV(float(s_IBD[8:14].strip()))
                f_Delay_pe = fnt_pe2MeV(float(s_IBD[36:42].strip()))
                i_Multiplicidade = int(s_IBD[64:70].strip())
                i_Multip_prompt = int(s_IBD[92:98].strip())
                i_Multip_delay = int(s_IBD[99:105].strip())
                if ((f_Time_ns > 0)and(i_Prompt_pe > 0)
                  and(i_Delay_pe > 0)and(i_Multiplicidade > 0)
                    and(f_Delay_pe < 20)):
                    xf.append(f_Time_ns)
                    pi.append(i_Prompt_pe)
                    di.append(i_Delay_pe)
                    pf.append(f_Prompt_pe)
                    df.append(f_Delay_pe)
                    mf.append(i_Multiplicidade)
                    mp.append(i_Multip_prompt)
                    md.append(i_Multip_delay)
                if ((f_Time_ns >= 8000)and(f_Time_ns <= 50000)
                    and(f_Prompt_pe >= 3)and(f_Prompt_pe <= 10)
                    and(f_Delay_pe >= 1.6)and(f_Delay_pe <= 7.6)
                    and(i_Multip_delay >= 25)):
                    nf.append(fnt_Positron2Neutrino(f_Prompt_pe))
        fo.close()

    return([xf,pi,di,mf,pf,df,nf,mp,md,nr_eventos])

##### Memo1.Lines.Add(Format('%6d %6d %20.9f %6d %20.9f %6d %20.9f %6d %6d %6d %6d %6d %6d %6d',
#####  [Event, i_Prompt_Hits, f_Prompt_Time, i_Delay_Hits, f_Delay_Time, Multiplicidade, f_Delay_Time - f_Prompt_Time,
#####   Multip_Prompt, Multip_Delay, i_Prompt_Hits_Veto, i_Delay_Hits_Veto, Multip_Prompt_Veto, Multip_Delay_Veto, Multiplicidade_Veto]));

In [4]:
def fnt_array_IBD_VT(list_sum):  #TV by target and veto - 04/03/2025
    xf = []
    pi = []
    di = []
    pf = []
    df = []
    mf = []
    mp = []
    md = []
    nf = []
    vpi = []
    vdi = []
    vpdi = []
    vtpi_ap = []
    vtdi_ap = []
    vtpi_ad = []
    vtdi_ad = []
    vtpi_apd = []
    vtdi_apd = []
    nr_eventos = 0
    exclude_delay = 12 ## exclude delay above 12pe 
    for filetr in list_sum:
        with open(filetr) as fo:
            s_IBD = fo.readline().replace(",",".")
            nr_eventos += 1
            i_Prompt_pe = int(s_IBD[8:14].strip())   #prompt in target
            f_Prompt_pe = fnt_pe2MeV(float(s_IBD[8:14].strip()))
            i_Delay_pe = int(s_IBD[36:42].strip())   #delay in target
            f_Delay_pe = fnt_pe2MeV(float(s_IBD[36:42].strip()))
            i_Multiplicidade = int(s_IBD[64:70].strip())
            f_Time_ns = float(s_IBD[71:91].strip())  #f_Delay_Time - f_Prompt_Time
            i_Multip_prompt = int(s_IBD[92:98].strip())
            i_Multip_delay = int(s_IBD[99:105].strip())
            i_V_Prompt_pe = int(s_IBD[106:112].strip())  #prompt in veto
            f_V_Prompt_pe = fnt_pe2MeV(float(s_IBD[106:112].strip()))
            i_V_Delay_pe = int(s_IBD[113:119].strip())   #delay in veto
            f_V_Delay_pe = fnt_pe2MeV(float(s_IBD[113:119].strip()))
            i_V_Multiplicidade = int(s_IBD[134:140].strip())

            if ((f_Time_ns >= 0)and(i_Multiplicidade+i_V_Multiplicidade > 0)):
                xf.append(f_Time_ns)
                # register if there are prompt and delay only in the target
                if ((i_Prompt_pe > 0)and(i_Delay_pe > 0)and(f_Delay_pe < exclude_delay)
                  and(i_V_Prompt_pe == 0)and(i_V_Delay_pe == 0)):
                    if (i_Multip_prompt>=5):
                        pi.append(i_Prompt_pe)
                        pf.append(f_Prompt_pe)
                    if (i_Multip_delay>=5):
                        di.append(i_Delay_pe)
                        df.append(f_Delay_pe)
                    mf.append(i_Multiplicidade)
                    mp.append(i_Multip_prompt)
                    md.append(i_Multip_delay)
                # register if there are prompt in the veto
                if (i_V_Prompt_pe > 0):
                    vpi.append(i_V_Prompt_pe)
                # register if there are delay in the veto
                if (i_V_Delay_pe > 0)and(f_V_Delay_pe < exclude_delay):
                    vdi.append(i_V_Delay_pe)
                # register if there are delay in the veto
                if (i_V_Prompt_pe > 0)and(i_V_Delay_pe > 0)and(f_V_Delay_pe < exclude_delay):
                    vpdi.append(i_V_Prompt_pe+i_V_Delay_pe)
                # register if there are prompt in the veto or target and delay in the target
                if (i_Prompt_pe+i_V_Prompt_pe > 0)and(i_Delay_pe > 0)and(i_V_Delay_pe == 0)and(f_Delay_pe < exclude_delay):
                    vtpi_ap.append(i_Prompt_pe+i_V_Prompt_pe) # ap - all prompt
                    vtdi_ap.append(i_Delay_pe)
                # register if there are delay in the veto or target and prompt in the target
                if (i_Delay_pe+i_V_Delay_pe > 0)and(i_Prompt_pe > 0)and(i_V_Prompt_pe == 0)and(f_Delay_pe+f_V_Delay_pe < exclude_delay):
                    vtpi_ad.append(i_Prompt_pe)
                    vtdi_ad.append(i_Delay_pe+i_V_Delay_pe) #ad - all delay
                # register if there are delay in the veto or target and prompt in the target
                if (i_Delay_pe+i_V_Delay_pe > 0)and(i_Prompt_pe+i_V_Prompt_pe > 0)and(f_Delay_pe+f_V_Delay_pe < exclude_delay):
                    vtpi_apd.append(i_Prompt_pe+i_V_Prompt_pe)  #apd - all delay and prompt
                    vtdi_apd.append(i_Delay_pe+i_V_Delay_pe)
                
            if ((f_Time_ns >= 8000)and(f_Time_ns <= 50000)
              and(f_Prompt_pe >= 3)and(f_Prompt_pe <= 10)
              and(i_Multip_prompt >=5)
              and(f_Delay_pe >= 1.6)and(f_Delay_pe <= 7.6)
              and(i_Multip_delay >= 25)and((i_V_Prompt_pe == 0)and(i_V_Delay_pe == 0))):
                nf.append(fnt_Positron2Neutrino(f_Prompt_pe))
        
            while (s_IBD.strip() != ""):
                s_IBD = fo.readline().replace(",",".")
                if (s_IBD.strip() == ""):
                    break
                nr_eventos += 1
                #print('IBD: '+s_IBD)
                #print('IBD p: '+s_IBD[8:14].strip())
                f_Time_ns = float(s_IBD[71:91].strip())
                i_Prompt_pe = int(s_IBD[8:14].strip())
                i_Delay_pe = int(s_IBD[36:42].strip())
                f_Prompt_pe = fnt_pe2MeV(float(s_IBD[8:14].strip()))
                f_Delay_pe = fnt_pe2MeV(float(s_IBD[36:42].strip()))
                i_Multiplicidade = int(s_IBD[64:70].strip())
                i_Multip_prompt = int(s_IBD[92:98].strip())
                i_Multip_delay = int(s_IBD[99:105].strip())
                i_V_Prompt_pe = int(s_IBD[106:112].strip())  #prompt in veto
                f_V_Prompt_pe = fnt_pe2MeV(float(s_IBD[106:112].strip()))
                i_V_Delay_pe = int(s_IBD[113:119].strip())   #delay in veto
                f_V_Delay_pe = fnt_pe2MeV(float(s_IBD[113:119].strip()))
                i_V_Multiplicidade = int(s_IBD[134:140].strip())
            
                if ((f_Time_ns >= 0)and(i_Multiplicidade+i_V_Multiplicidade > 0)):
                    xf.append(f_Time_ns)
                    # register if there are prompt and delay only in the target
                    if ((i_Prompt_pe > 0)and(i_Delay_pe > 0)and(f_Delay_pe < exclude_delay)
                       and(i_V_Prompt_pe == 0)and(i_V_Delay_pe == 0)):
                        if (i_Multip_prompt>=5):
                            pi.append(i_Prompt_pe)
                            pf.append(f_Prompt_pe)
                        if (i_Multip_delay>=5):
                            di.append(i_Delay_pe)
                            df.append(f_Delay_pe)
                        mf.append(i_Multiplicidade)
                        mp.append(i_Multip_prompt)
                        md.append(i_Multip_delay)
                    # register if there are prompt in the veto
                    if (i_V_Prompt_pe > 0):
                        vpi.append(i_V_Prompt_pe)
                    # register if there are delay in the veto
                    if (i_V_Delay_pe > 0)and(f_V_Delay_pe < exclude_delay):
                        vdi.append(i_V_Delay_pe)
                    # register if there are delay in the veto
                    if (i_V_Prompt_pe > 0)and(i_V_Delay_pe > 0)and(f_V_Delay_pe < exclude_delay):
                        vpdi.append(i_V_Prompt_pe+i_V_Delay_pe)
                    # register if there are prompt in the veto or target and delay in the target
                    if (i_Prompt_pe+i_V_Prompt_pe > 0)and(i_Delay_pe > 0)and(i_V_Delay_pe == 0)and(f_Delay_pe < exclude_delay):
                        vtpi_ap.append(i_Prompt_pe+i_V_Prompt_pe) # ap - all prompt
                        vtdi_ap.append(i_Delay_pe)
                    # register if there are delay in the veto or target and prompt in the target
                    if (i_Delay_pe+i_V_Delay_pe > 0)and(i_Prompt_pe > 0)and(i_V_Prompt_pe == 0)and(f_Delay_pe+f_V_Delay_pe < exclude_delay):
                        vtpi_ad.append(i_Prompt_pe)
                        vtdi_ad.append(i_Delay_pe+i_V_Delay_pe) #ad - all delay
                    # register if there are delay in the veto or target and prompt in the target
                    if (i_Delay_pe+i_V_Delay_pe > 0)and(i_Prompt_pe+i_V_Prompt_pe > 0)and(f_Delay_pe+f_V_Delay_pe < exclude_delay):
                        vtpi_apd.append(i_Prompt_pe+i_V_Prompt_pe)  #apd - all delay and prompt
                        vtdi_apd.append(i_Delay_pe+i_V_Delay_pe)

                if ((f_Time_ns >= 8000)and(f_Time_ns <= 50000)
                  and(f_Prompt_pe >= 3)and(f_Prompt_pe <= 10)
                  and(i_Multip_prompt >=5)
                  and(f_Delay_pe >= 1.6)and(f_Delay_pe <= 7.6)
                  and(i_Multip_delay >= 25)and((i_V_Prompt_pe == 0)and(i_V_Delay_pe == 0))):
                    nf.append(fnt_Positron2Neutrino(f_Prompt_pe))
        fo.close()

    return([xf,pi,di,mf,pf,df,nf,mp,md,nr_eventos, 
            vpi,vdi,vpdi,vtpi_ap,vtdi_ap,vtpi_ad,vtdi_ad,vtpi_apd,vtdi_apd])